For PacBio I need to set uop the analysis rather differently
I used the following tutorial to do alignments and filtering
https://jbloomlab.github.io/alignparse/lasv_pilot.html
Unfortunately it doesn't work with no filtering so I just tried to set the filtering very loose
That resulted in ~10% of reads being filtered, they are likely the "worst" 10%
Jack suggested using consensus to figure out what mutations are real
 For barcodes with >2 reads we can just check if associated mutations appear in most of the barcode reads
 Only accept mutations that appear in a majority (>90%?)
Then we can filter on there being just one, allowed mutation
At that point we can basically move on to statistics and use the same ones generated in NP-11-19 and 24

Add backbone stuff in


In [6]:
import os
import tempfile
import warnings

import Bio.SeqIO

import pandas as pd
import numpy

from plotnine import *

import pysam

import alignparse.ccs
import alignparse.minimap2
import alignparse.targets
import alignparse.cs_tag
import alignparse.consensus

import matplotlib
import matplotlib.pyplot as plt

In [7]:
# First I make a template based on the plasmid with only the features we care about

with open('np_11_21_1_bb.gb') as f:
    print(f.read())

LOCUS       NP_11_21_1              5719 bp ds-DNA     circular     07-MAY-2021
DEFINITION  .
KEYWORDS    "Personalized Plasmid ID" "Plasmid Name" "Description" "Selection" 
            "Concentration (ng/uL)" "Location" "Sequenced verified?" "Date
            Added" "Glycerol Stock?"
FEATURES             Location/Qualifiers
     gene            1..1404
                     /label="rbcLCodonOpt"
     CDS             1..1404
                     /label="Translation 1-1404"
     Barcode         1426..1440
                     /label="Barcode"
     backbone        1441..5719
                     /label="backbone"
ORIGIN
        1 ATGGACCAGA GTTCTCGCTA TGTCAATCTT GCTTTGAAAG AAGAGGACTT AATTGCCGGA
       61 GGCGAACACG TATTGTGCGC CTATATTATG AAACCAAAAG CTGGGTACGG TTACGTTGCC
      121 ACCGCAGCGC ACTTTGCTGC AGAGAGTTCG ACTGGAACAA ATGTAGAGGT GTGTACCACT
      181 GATGATTTTA CTCGTGGCGT GGATGCCTTA GTTTACGAAG TAGATGAGGC CCGCGAGCTT
      241 ACTAAGATTG CCTACCCAGT AGCACTGTTC GATCGTAATA TCACGGATGG AAAAGC

In [9]:
# The Bloom lab uses something called a YAML file that tells the aligner how to parse features
# I edited the one they use for Lassa to fit rbcL, this will return mutations in rbcL and the sequence of the barcode

parse_specs_file = 'rbcLDMSbb.yaml'
    
targets = alignparse.targets.Targets(seqsfile = 'np_11_21_1_bb.gb', \
                                     feature_parse_specs = parse_specs_file, \
                                     allow_extra_features = True, \
                                     allow_clipped_muts_seqs = True)

print(targets.feature_parse_specs('yaml'))


NP_11_21_1:
  query_clip5: null
  query_clip3: null
  backbone:
    return:
    - mutations
    - accuracy
    filter:
      mutation_nt_count: null
      mutation_op_count: null
      clip5: null
      clip3: null
  gene:
    filter:
      mutation_nt_count: null
      mutation_op_count: null
      clip5: null
      clip3: null
    return:
    - mutations
    - accuracy
  Barcode:
    return:
    - sequence
    filter:
      clip5: 0
      clip3: 0
      mutation_nt_count: 0
      mutation_op_count: 0



In [10]:
# This chooses minimap2 as the aligner

mapper = alignparse.minimap2.Mapper(alignparse.minimap2.OPTIONS_CODON_DMS, \
                                    prog='minimap2-2.17-hbbe82c9_3/bin/minimap2')

# It's possible to do multiple targets and have different categories of files 
# This just does one queryfile (the fastq file with the data) and one template ('targets' defined above)
# The output is a SAM file

targets.align(queryfile="/Volumes/NP_DFS_4TB/NP_11_25/subreads_ccs.fastq", \
              alignmentfile='rbcLalignmentsBB.sam', \
              mapper=mapper)


In [11]:
# This figures out what aligned and what was filtered out

run_readstats, run_aligned, run_filtered = targets.parse_alignment('rbcLalignmentsBB.sam', filtered_cs=False)
run_readstats

,category,count
0,aligned NP_11_21_1,3585880
1,filtered NP_11_21_1,184045
2,unmapped,79734


In [12]:
# here we find out why stuff was filtered

run_filtered['NP_11_21_1']


,query_name,filter_reason
0,m64044_201203_021636/1/ccs,Barcode clip5
1,m64044_201203_021636/14/ccs,Barcode mutation_nt_count
2,m64044_201203_021636/47/ccs,Barcode clip5
3,m64044_201203_021636/72/ccs,Barcode clip5
4,m64044_201203_021636/101/ccs,Barcode mutation_nt_count
...,...,...
184040,m64044_201203_021636/180554275/ccs,Barcode clip5
184041,m64044_201203_021636/180554276/ccs,Barcode mutation_nt_count
184042,m64044_201203_021636/180554327/ccs,Barcode clip5
184043,m64044_201203_021636/180554512/ccs,Barcode clip5


In [13]:
# Here we can see mutations and barcodes and make a csv file for future use



# Change filename to have automated experiment name added
run_aligned['NP_11_21_1'].to_csv('NP_11_21_1_backboneAware.csv')

run_aligned['NP_11_21_1']



,query_name,query_clip5,query_clip3,backbone_mutations,backbone_accuracy,gene_mutations,gene_accuracy,Barcode_sequence
0,m64044_201203_021636/2/ccs,2404,0,del173to173 T1874C C1875T,0.999810,ins837C C1130G C1131T,0.999530,TAAATGTTAAGGCGT
1,m64044_201203_021636/4/ccs,2384,0,del62to62 ins88G del127to127 ins207C ins327T d...,0.988948,del15to15 G36T A37G C57G G58C ins96A del212to2...,0.984765,AACACTTAACATCTG
2,m64044_201203_021636/5/ccs,2410,0,ins52G ins72C A281C ins372C ins392A del406to40...,0.993817,ins103G ins302T ins385C ins523C del730to730 A1...,0.993166,CTGCCCTCAATTAGA
3,m64044_201203_021636/6/ccs,2418,0,ins178A ins318C ins334C ins345C ins434T ins623...,0.997527,ins96A ins119C ins198C ins448C ins837CC ins868...,0.997315,TTAATAAGAGAAATG
4,m64044_201203_021636/7/ccs,2404,0,,0.999866,C523G C524A T525A G645A,1.000000,AGAATCTACAATCCC
...,...,...,...,...,...,...,...,...
3585875,m64044_201203_021636/180554585/ccs,2405,0,,0.999731,C101A T102A,0.999928,TCTGGGGTGATGGGG
3585876,m64044_201203_021636/180554586/ccs,2404,0,del349to349,0.999894,C265T T266G,1.000000,TCTTTAGGTCCACGT
3585877,m64044_201203_021636/180554587/ccs,2394,0,del120to120 del446to446 del480to480 del578to57...,0.997998,ins124G G295T C296G T297C del564to564 del666to...,0.997055,TATTCTCCTTCCTTT
3585878,m64044_201203_021636/180554589/ccs,2394,0,del96to96 C198G G200T del222to222 ins761G del1...,0.997614,del80to80 del255to255 del291to291 del327to327 ...,0.996862,GCTTGCTGAATAGCC
